Nome: Daniel Oliveira dos Santos              
Matrícula: 564307                
Curso: CIÊNCIA DA COMPUTAÇÃO                     
Disciplina: AVALIAÇÃO DE DESEMPENHO DE SISTEMAS COMPUTACIONAIS                      
Atividade: Gerar Curva ROC.

Classificando e-mail como spam ou não spam

Características do conjunto de dados
Multivariada

Área da matéria
Ciência da Computação

Tarefas Associadas
Classificação

Tipo de recurso
Inteiro, Real

# Instâncias
4601

# Características
57

link: https://archive.ics.uci.edu/dataset/94/spambase

# **Bibliotecas**

In [ ]:
# Irei usar dois modelo Regressão Logistica


# Bibliotecas

# Gráficos
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_palette("Accent")
sns.set_style("darkgrid")
import plotly.express as px
import plotly.graph_objects as go


# Tratamento de dados
from sklearn.model_selection import cross_val_score, cross_val_predict
import numpy as np
import pandas as pd

# Tratamento de dados para separar treino, teste e normalização
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold

# Modelo
from sklearn.linear_model import LogisticRegression

# Métrica
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.metrics import roc_auc_score, roc_curve, classification_report,\
                            accuracy_score, confusion_matrix, auc




# **DATASET**

In [ ]:
# baixando os dados
!pip install ucimlrepo

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
spambase = fetch_ucirepo(id=94)

# data (as pandas dataframes)
X = spambase.data.features
y = spambase.data.targets

# metadata
#print(spambase.metadata)

# variable information
#print(spambase.variables)
y = np.array(y)

In [ ]:
X.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.0,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.0,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.0,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.0,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.0,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191


ATRIBUTOS DE BANCO DE DADOS DE E-MAIL DE SPAM (no formato .names)
|
| 48 atributos reais contínuos [0,100] do tipo word_freq_WORD           
| = porcentagem de palavras no e-mail que correspondem a WORD,           
| ou seja, 100 * (número de vezes que a PALAVRA aparece no e-mail) /    
| número total de palavras no e-mail. Uma "palavra" neste caso é qualquer   
| sequência de caracteres alfanuméricos delimitados por caracteres não alfanuméricos                   
| caracteres ou fim de string.                
|                                                                           
| 6 atributos reais contínuos [0,100] do tipo char_freq_CHAR               
| = porcentagem de caracteres no e-mail que correspondem a CHAR,                     
| ou seja, 100 * (número de ocorrências CHAR)/total de caracteres no e-mail   
|                                     
| 1 atributo real contínuo [1,...] do tipo capital_run_length_average                    
| = comprimento médio de sequências ininterruptas de letras maiúsculas         
|                                                             
| 1 atributo inteiro contínuo [1,...] do tipo capital_run_length_longest               
| = comprimento da maior sequência ininterrupta de letras maiúsculas      
|                           
| 1 atributo inteiro contínuo [1,...] do tipo capital_run_length_total      
| = soma do comprimento das sequências ininterruptas de letras maiúsculas                  
| = número total de letras maiúsculas no e-mail                                
|                                                                
| 1 atributo de classe nominal {0,1} do tipo spam              
| = denota se o e-mail foi considerado spam (1) ou não (0),             
| ou seja, e-mail comercial não solicitado.                                
|                                                          
| Para obter mais informações, consulte o arquivo 'spambase.DOCUMENTATION' no
| Repositório de aprendizado de máquina UCI: http://www.ics.uci.edu/~mlearn/MLRepository.html

Preparando os dados

# Usando o Modelo regressão logistica da biblioteca sklearn

In [ ]:
# Defina o modelo_logistico e os dados X e y antes de prosseguir
# Inicialize o modelo de regressão logística
modelo_logistico = LogisticRegression(solver = 'lbfgs', multi_class = 'auto', max_iter = 1000)

# modelo_logistico'seja seu modelo treinado e 'X_normalizando' e 'y' sejam seus dados normalizados e seus rótulos
scaler = MinMaxScaler()
X_normalizado = scaler.fit_transform(X)


# Separando os dados de treino e testes a 30 porcento de teste e 80 porcento de treino
X_treino, X_teste, y_treino, y_teste =  train_test_split(X_normalizado,y, test_size=0.30, random_state=42)


# Treinar o modelo
modelo_logistico.fit(X_treino, y_treino)


# Calcular probabilidades previstas para os dados de teste
probabilidades_previstas = modelo_logistico.predict_proba(X_teste)[:, 1]


# Calcular a pontuação de validação para o fold atual
# score = modelo_logistico.score(X_teste, y_teste)

# Prevendo o modelo
y_previsto = modelo_logistico.predict(X_treino)

# Calcular a curva ROC para o fold atual
fpr, tpr, _ = roc_curve(y_teste, probabilidades_previstas)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Métricas usando a biblioteca

In [ ]:
# Calcular a acurácia
acuracia = accuracy_score(y_treino, y_previsto).round(4)
print("Acurácia do modelo:", acuracia)


Acurácia do modelo: 0.8826


# **Gráficos**

In [ ]:
# Gráfico de rosca para métricas

def grafico_rosca(nome_metrica,valor_da_metrica,cor):
  # Valor da acurácia
  valor = valor_da_metrica * 100


  # Calcule o tamanho da "rosca"
  rosca_size = 100 - valor

  # Crie um DataFrame com os dados
  data = {'Categoria': [f'{nome_metrica}', ''],
          'Valor': [valor, rosca_size]}
  df = pd.DataFrame(data)

  # Crie o gráfico de pizza com Plotly Express
  fig = px.pie(df, values='Valor',
              names='Categoria',
              hole=0.7,
              color_discrete_sequence=[f'{cor}', 'lightgray'])


  fig.update_traces(textinfo='none',
                    showlegend=False,
                    #marker=dict(line=dict(color='cor', width=2))
                    )

  fig.update_layout(#title='Acurácia',
                    width=650,
                    height=450,
                    )

  fig.add_annotation(x=0.5, y=1.15, text=f'<b>{nome_metrica}</b>', font=dict(color = cor, size=29, family='Arial, sans-serif'), showarrow=False)
  # Adicione o valor da acurácia no centro
  fig.add_annotation(x=0.5, y=0.5, text=f'<b>{valor:.2f}%</b>',font=dict(color= cor, size=32, family='Arial, sans-serif'),  showarrow=False)

  return fig.show()

In [ ]:
grafico_rosca('Acuracia',acuracia,'navy')

In [ ]:
# Matrix de confusão
matriz_confusao = confusion_matrix(y_treino, modelo_logistico.predict(X_treino)) # usando a função do sklearn
matriz_confusao

array([[1884,  100],
       [ 278,  958]])

In [ ]:
# Converte a matriz de confusão em um DataFrame para plotagem
df_matriz_confusao = pd.DataFrame(matriz_confusao, columns=np.unique(y_treino), index=np.unique(y_treino))

In [130]:
import plotly.graph_objects as go
# Cria o mapa de calor
fig = px.imshow(df_matriz_confusao,
                labels=dict(x="Valor Prevista", y="Valor Real", color="Contagem"),
                x=df_matriz_confusao.columns,  # Rótulos para o eixo x
                y=df_matriz_confusao.index,  # Rótulos para o eixo y
                color_continuous_scale="pubu",  # Escolhe uma escala de cores
                )

# Adiciona anotações com os valores
fig.update_traces(text=df_matriz_confusao.values, selector=dict(type='heatmap'))



# Adiciona os números dentro do mapa de calor
for i in range(len(df_matriz_confusao.index)):
    for j in range(len(df_matriz_confusao.columns)):

        valor = df_matriz_confusao.iloc[i, j]
        # Texto para cada quadrado
        if((i == 0) and (j == 0)):
            texto = f"Verdadeiro Positivo:<br> {valor}"  # Verdadeiro Positivo
        elif((i == 1) and (j == 1)):
            texto = f"Verdadeiro Negativo:<br> {valor}"  # Verdadeiro Negativo

        elif((i == 0) and (j == 1)):
            texto = f"Falso Negativo:<br> {valor}"  # Falso Negativo

        elif((i == 1) and (j == 0)):
            texto = f"Falso Positivo:<br> {valor}"  # Falso Positivo

        # Adiciona o texto ao gráfico
        fig.add_annotation(
            text=texto,
            x=df_matriz_confusao.columns[j],
            y=df_matriz_confusao.index[i],
            showarrow=False,
            font=dict(color='rgb(0,0,128)', size=18),  # Definindo a cor do texto
            xshift=10,  # Ajusta a posição horizontal do texto
            yshift=10,  # Ajusta a posição vertical do texto
        )

# Personaliza o layout
fig.update_layout(
    title="Matriz de Confusão",
    xaxis_title="Classe Prevista",
    yaxis_title="Classe Real",
    margin=dict(l=50, r=350, t=50, b=20),  # Define as margens
    width=1000,
    height=600,
)



# Adicionar texto ao lado do gráfico
fig.add_annotation(
    text=f'<b>Observação</b><br>É uma medida de<br> desempenho de um modelo<br> preditivo de Machine Learning.<br>    \
     Comum na resolução<br> de problemas<br> binários (duas classes),<br> formada por duas<br> colunas e duas linhas. ',
    xref='paper',  # Posicionamento em relação ao papel (gráfico)
    yref='paper',  # Posicionamento em relação ao papel (gráfico)
    x=1.58,  # Posição horizontal do texto
    y=0.85,   # Posição vertical do texto
    showarrow=False,  # Não exibir seta
    font=dict(size=16),  # Tamanho da fonte do texto
    bgcolor='lightblue',  # Cor de fundo azul claro
    #bordercolor='blue',  # Cor da borda
    borderwidth=2,  # Largura da borda

)


# Mostra o mapa de calor
fig.show()

# Curva ROC usando a biblioteca

In [ ]:
# modelo de regressão logística chamado modelo
probabilidades = modelo_logistico.predict_proba(X_teste)

# A probabilidade da classe positiva (classe 1) é a segunda coluna
probabilidade_classe_1 = probabilidades[:, 1]


# Calcula a curva ROC
fpr, tpr, thresholds = roc_curve(y_teste, probabilidade_classe_1)

# Calcula a área sob a curva ROC (AUC)
auc = roc_auc_score(y_teste, probabilidade_classe_1)

In [25]:
fig = px.area(
            x=fpr,  # Índice do DataFrame como eixo x
            y=tpr,  # Colunas do DataFrame como eixo y
            color_discrete_sequence=px.colors.qualitative.Dark24,  # Sequência de cores
            labels={'x': 'Índice', 'y': 'Valor'},  # Rótulos dos eixos
            title='<b>Curva ROC</b>',

)


# Adicionar a linha diagonal de referência
fig.add_shape(
    type='line',
    x0=0,
    y0=0,
    x1=1,
    y1=1,
    line=dict(color='black', dash='dash'),
)

# Adicionar texto ao lado do gráfico
fig.add_annotation(
    text=f'<b>Área da curva</b> <br>Área da curva é o<br>valor da acurácia<br><br>{acuracia*100:.2f}%',
    xref='paper',  # Posicionamento em relação ao papel (gráfico)
    yref='paper',  # Posicionamento em relação ao papel (gráfico)
    x=1.30,  # Posição horizontal do texto
    y=0.8,   # Posição vertical do texto
    showarrow=False,  # Não exibir seta
    font=dict(size=20),  # Tamanho da fonte do texto
    bgcolor='lightblue',  # Cor de fundo azul claro
    #bordercolor='blue',  # Cor da borda
    borderwidth=2,  # Largura da borda
)


# Atualizar o layout do gráfico
fig.update_layout(
    xaxis_title='Taxa de Falso Positivo (FPR)<br>(ou 1 — Especificidade)',  # Título do eixo x
    yaxis_title='Taxa de Verdadeiros Positivos<br>(TPR) (ou Sensibilidade/Recall)',  # Título do eixo y
    legend_title= acuracia,  # Título da legenda
    margin=dict(l=50, r=200, t=50, b=20),  # Define as margens
    yaxis=dict(showgrid=True, zeroline=False),  # Exibir grade no eixo y
    width=900,  # Largura do gráfico
    height=500  # Altura do gráfico
)

# Mostrar o gráfico
fig.show()

# Métricas manual

In [ ]:
# Funções de métricas


class Metricas:

  def __init__(self,real,previsto):

     self.real = real.T

     self.previsto = previsto

     # Total
     self._total = len(self.real)

     # Verdadeiros Positivos (VP): São as instâncias que são positivas e que o modelo classificou corretamente como positivas.
     self._VP = np.sum((self.real == 1) & (self.previsto == 1))

     # Verdadeiros Negativos (VN): São as instâncias que são negativas e que o modelo classificou corretamente como negativas.
     self._VN = np.sum((self.real == 0) & (self.previsto == 0))

    # Falsos Positivos (FP): São as instâncias que são negativas, mas que o modelo classificou erroneamente como positivas.
     self._FP = np.sum((self.real == 0) & (self.previsto == 1))

    # Falsos Negativos (FN): São as instâncias que são positivas, mas que o modelo classificou erroneamente como negativas.
     self._FN = np.sum((self.real == 1) & (self.previsto == 0))



  def acuracia(self):
     acuracia = (self._VP + self._VN) / (self._VP + self._VN + self._FP + self._FN)
     return acuracia

  def precisao(self):
    precision = self._VP / (self._VP + self._FP)
    return precision

  def recall(self):
    recall = self._VP / (self._VP + self._FN)
    return recall

  def especificidade(self):
    taxa = self._VN / (self._VN + self._FP)
    return taxa

  def f1_score(self):
    score = 2 * ((self.precisao() * self.recall()) / (self.precisao() + self.recall()))
    return score

  def formatacao(self, nome):
    if nome == 'f1_score':
      return f'{self.f1_score() * 100:.2f}'
    if nome == 'especificidade':
      return  f'{self.especificidade() * 100:.2f}'
    if nome == 'recall':
      return  f'{self.recall() * 100:.2f}'
    if nome == 'precisao':
      return  f'{self.precisao() * 100:.2f}'
    if nome == 'acuracia':
      return  f'{self.acuracia() * 100:.2f}'

  @property
  def VP(self):
        # Este código é executado quando alguém for
        # ler o valor de self.nome
        return self._VP

  @property
  def VN(self):
        return self._VN

  @property
  def FP(self):
        return self._FP

  @property
  def FN(self):
        return self._FN



In [ ]:
metrics = Metricas(y_treino, y_previsto)

In [ ]:
matrix_de_confusao= [metrics.VN,metrics.FP,metrics.FN,metrics.VP]
matrix_de_confusao

[1884, 100, 278, 958]

In [ ]:
print(f"Acuracia:.........{metrics.formatacao('acuracia')}%")
print(f"especificidade:...{metrics.formatacao('especificidade')}%")
print(f"recall:...........{metrics.formatacao('recall')}%")
print(f"precisao:.........{metrics.formatacao('precisao')}%")
print(f"f1_score:.........{metrics.formatacao('f1_score')}%")

Acuracia:.........88.26%
especificidade:...94.96%
recall:...........77.51%
precisao:.........90.55%
f1_score:.........83.52%


In [ ]:
acuracia = metrics.acuracia().round(4)
grafico_rosca('Acuracia',acuracia,'rgb(222,45,38)')

In [ ]:
recall = metrics.recall().round(4)
grafico_rosca('Recall',recall,'rgb(197,27,138)')

In [ ]:
especificidade = metrics.especificidade().round(4)
grafico_rosca('Especificidade',especificidade,'rgb(28,144,153)')

In [ ]:
precisao = metrics.precisao().round(4)
grafico_rosca('Precisão',precisao,'rgb(117,107,177)')

In [ ]:
f1_score = metrics.f1_score().round(4)
grafico_rosca('F1 score',f1_score,'rgb(44,162,95)')

In [ ]:
metrics.f1_score().round(4)

0.8352